In [1]:
# 의존성 패키지 설치
!pip install ratsnlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 KB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 582.5/582.5 KB 44.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 KB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 68.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.4/512.4 KB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 KB 43.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=1d5cde1e56fa5d682007a21375f61aea92e6946597971b55125927dbcc65d094
  Stored in directory: /root/.cache/pip/wheels/82/ab/9b/c1

## 구글 드라이브 연동하기

In [2]:
# 구글 드라이브 연동
from google.colab import drive
drive.mount('/gdrive',force_remount=True)

Mounted at /gdrive


## 각종 설정

모델 하이퍼파라메터(hyperparameter)와 저장 위치 등 설정 정보를 선언

인자(argument)의 역할과 내요
- pretrained_model_name : 이전 장에서 파인튜닝한 모델이 사용한 프리트레인 마친 언어모델 이름(단 해당 모델은 허깅페이스 라이브러리에 등록되어 있어야 합니다)
- downstream_model_dir : 이전 장에서 파인튜닝한 모델의 체크포인트 저장 위치.
- max_seq_length : 토큰 기준 입력 문장 최대 길이. 아무 것도 입력하지 않으면 128입니다.


In [4]:
# 인퍼런스 설정
from ratsnlp.nlpbook.classification import ClassificationDeployArguments
args=ClassificationDeployArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_model_dir="/gdrive/My Drive/nlpbook/checkpoint-doccls1",
    max_seq_length=128
)

downstream_model_checkpoint_fpath: /gdrive/My Drive/nlpbook/checkpoint-doccls1/epoch=0-val_loss=0.27.ckpt


## 모델 로딩

파인튜닝을 마친 모델과 토크나이저 읽어 들입니다.

In [6]:
# 토크나이저 로드
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False
)

Downloading:   0%|          | 0.00/250k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

In [5]:
import torch
from transformers import BertConfig, BertForSequenceClassification
# 체크포인트 로드
fine_tuned_model_ckpt=torch.load(
    args.downstream_model_checkpoint_fpath,
    map_location=torch.device("cpu")
)
# 파인튜닝한 모델이 사용한 프리트레인 마친 언어모델의 설정 값들을 읽어 들일 수 있음
pretrained_model_config=BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels=fine_tuned_model_ckpt['state_dict']['model.classifier.bias'].shape.numel()
)
# 초기화한 BERT 모델에 체크포인트(fine_tuned_model_ckpt)를 읽어들이게 된다
model=BertForSequenceClassification(pretrained_model_config)
model.load_state_dict({k.replace("model.",""): v for k, v in fine_tuned_model_ckpt['state_dict'].items()})  # 체크포인트 주입
# 모델이 평가 모드로 전환. 드롭아웃 등 학습 때만 사용하는 기법들을 무효화하는 역할을 한다
model.eval()

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

## 인퍼런스 함수 선언
- 문장(sentence)에 토큰화를 수행한 뒤 input_ids, attention_mask, token_type_ids를 만든다. 
- 이들 입력값을 파이토치 텐서(tensor) 자료형으로 변환한 뒤 모델에 입력합니다. - 모델 출력 값(outputs.logits)은 소프트맥스 함수 적용 이전의 로짓(logit) 형태인데요. 여기에 소프트맥스 함수를 써서 모델 출력을 [부정일 확률, 긍정일 확률] 형태의 확률 형태로 바꾼다.
- 마지막으로 모델 출력을 약간 후처리하여 예측 확률의 최댓값이 부정 위치일 경우 해당 문장이 부정(positive), 반대의 경우 긍정(positive)이 되도록 pred 값을 만든다.


In [7]:
# 인퍼런스 함수
def inference_fn(sentence):
    # 문장을 토큰화한 뒤, input_id, attention_masks, token_type_ids 만들기
    inputs=tokenizer(
        [sentence],
        max_length=args.max_seq_length,
        padding='max_length',
        truncation=True
    )
    with torch.no_grad():
        # 모델 계산하기
        outputs=model(**{k: torch.tensor(v) for k, v in inputs.items()})    # inputs를 파이토치 텐서로 바꾸기
        prob=outputs.logits.softmax(dim=1)  # 로짓에 소프트맥스 취하기
        # 긍정/부정확률을 소수점 4자리로 반올림
        positive_prob=round(prob[0][1].item(),4)
        negative_prob=round(prob[0][0].item(),4)
        # 예측 확률의 최댓값 위치에 따라 pred만들기
        pred="긍정 (positive)" if torch.argmax(prob)==1 else "부정 (negative)"
    return {
        'sentence':sentence,
        'prediction':pred,
        'positive_data': f"긍정 {positive_prob}",
        'negative_data': f"부정 {negative_prob}",
        'positive_width': f"{positive_prob * 100}%",
        'negative_width': f"{negative_prob * 100}%"
    }

In [8]:
sentence="좋나요?"
inference_fn(sentence)

{'sentence': '좋나요?',
 'prediction': '긍정 (positive)',
 'positive_data': '긍정 0.9427',
 'negative_data': '부정 0.0573',
 'positive_width': '94.27%',
 'negative_width': '5.7299999999999995%'}

In [9]:
sentence="마지막에 아쉽네요."
inference_fn(sentence)

{'sentence': '마지막에 아쉽네요.',
 'prediction': '긍정 (positive)',
 'positive_data': '긍정 0.6903',
 'negative_data': '부정 0.3097',
 'positive_width': '69.03%',
 'negative_width': '30.97%'}

In [10]:
sentence="이건 뭔 내용이죠?"
inference_fn(sentence)

{'sentence': '이건 뭔 내용이죠?',
 'prediction': '부정 (negative)',
 'positive_data': '긍정 0.0065',
 'negative_data': '부정 0.9935',
 'positive_width': '0.65%',
 'negative_width': '99.35000000000001%'}

In [15]:
sentence="아쉬운듯 하면서 재미가 있는거 같기도하고 없는거 같기도하고 내용이 신기하면서도 애매하기도 하고 그런 느낌이네요"
inference_fn(sentence)

{'sentence': '아쉬운듯 하면서 재미가 있는거 같기도하고 없는거 같기도하고 내용이 신기하면서도 애매하기도 하고 그런 느낌이네요',
 'prediction': '부정 (negative)',
 'positive_data': '긍정 0.4376',
 'negative_data': '부정 0.5624',
 'positive_width': '43.76%',
 'negative_width': '56.24%'}

## 웹서비스 만들기 준비

`ngrok`은 코랩 로컬에서 실행 중인 웹서비스를 안전하게 외부에서 접근 가능하도록 해주는 도구입니다.  
`ngrok`을 실행하려면 [회원가입](https://dashboard.ngrok.com/signup) 후 [로그인](https://dashboard.ngrok.com/login)을 한 뒤 [이곳](https://dashboard.ngrok.com/get-started/your-authtoken)에 접속해 인증 토큰(authtoken)을 확인해야 합니다. 예를 들어 확인된 `authtoken`이 `test111`이라면 다음과 같이 실행합니다.

```bash
!mkdir /root/.ngrok2 && echo "authtoken: test111" > /root/.ngrok2/ngrok.yml
```


In [16]:
!mkdir /root/.ngrok2 && echo "authtoken: 2K7Hot53QiwZ1iwcLB0397k5jDK_4UCEFCawR6MqfXQNbFNxd" > /root/.ngrok2/ngrok.yml

In [18]:
!ls /root -al   # -al : .으로 저장된 데이터도 보여준다.

total 64
drwx------ 1 root root 4096 Jan 10 02:49 .
drwxr-xr-x 1 root root 4096 Jan 10 02:25 ..
-r-xr-xr-x 1 root root 1169 Jan  1  2000 .bashrc
drwxr-xr-x 1 root root 4096 Jan 10 02:25 .cache
drwx------ 1 root root 4096 Jan 10 02:24 .config
drwxr-xr-x 5 root root 4096 Jan  5 14:48 .ipython
drwx------ 2 root root 4096 Jan  5 14:48 .jupyter
drwxr-xr-x 2 root root 4096 Jan  5 14:45 .keras
drwxr-xr-x 1 root root 4096 Jan  5 14:48 .local
drwxr-xr-x 2 root root 4096 Jan 10 02:49 .ngrok2
drwxr-xr-x 4 root root 4096 Jan  5 14:48 .npm
-rw-r--r-- 1 root root  148 Aug 17  2015 .profile
-r-xr-xr-x 1 root root  254 Jan  1  2000 .tmux.conf
-rw-r--r-- 1 root root  165 Jan  5 14:48 .wget-hsts


## 웹서비스 개시

아래처럼 실행해 인퍼런스 함수를 웹서비스로 만든다.

In [19]:
# 웹서비스 - flask 파이썬 라이브러리 이용
from ratsnlp.nlpbook.classification import get_web_service_app
app=get_web_service_app(inference_fn)
app.run()

 * Serving Flask app "ratsnlp.nlpbook.classification.deploy" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://ef9b-34-86-43-157.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
